<a href="https://colab.research.google.com/github/delishadalu/MYGAME/blob/main/c138.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

# https://www.kaggle.com/tmdb/tmdb-movie-metadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
movies_df = pd.read_csv('/content/tmdb_5000_movies.csv')

In [3]:
credits_df = pd.read_csv('/content/tmdb_5000_credits.csv')

In [ ]:
credits_df.columns = ['id','title','cast','crew']
credits_df.head()

In [5]:
result_df = pd.merge(movies_df , credits_df , on = 'id')

In [ ]:
result_df.head()

c139
..
..
..


In [ ]:
result_df.info() # get all the coloumn names

In [ ]:
# clean up the data by deleting some coloumns out


result_df.drop(['homepage', 'title_x', 'title_y', 'production_companies'] , axis = 1 , inplace = True)

result_df.info()

In [ ]:
# detelete the coloumns having null values

result_df.dropna(inplace = True)
result_df.info()

In [10]:
# lets calculate the movie rating

R = result_df['vote_average']

C = result_df['vote_average'].mean()

v = result_df['vote_count']

m = result_df['vote_count'].quantile(0.9)

In [11]:
result_df['weighted_rating'] = (R*v + C*m) / (v + m)

In [ ]:
result_df[['original_title' , 'weighted_rating']].head()

In [13]:
import plotly.express as px
bar_plot = px.bar(result_df.head(10).sort_values('weighted_rating') , x = 'weighted_rating' , y = 'original_title' , orientation = 'h')
bar_plot.show()

c140
.
.
.


cast, crew(director) ,  keywords , genres are having difrrent labels in each

1. we want to take only the names out and create a list
....


..

In [ ]:
result_df[['original_title', 'cast', 'crew', 'keywords', 'genres']].head(3)

In [15]:
type(result_df.loc[0]['genres'])

str

In [16]:
# literal_eval() -->m helps to check what is the dataype and convert it to the list



from ast import literal_eval

In [17]:
features = ['cast' , 'crew' , 'keywords' , 'genres']

for feature in features:
  result_df[feature] = result_df[feature].apply(literal_eval)

In [18]:
type(result_df.loc[0]['cast']) # try for all features

list

In [ ]:
result_df.loc[0]['crew']

In [20]:
# create a function to get the director name
import numpy as np

def get_director(crew):
  for crew_member in crew:
    if crew_member['job'] == 'Director':
      return crew_member['name']
    
  return np.nan # not a number

In [21]:
# create a new heading director

result_df['director'] = result_df['crew'].apply(get_director)

In [ ]:
result_df[['original_title' , 'director']].head()

In [ ]:
result_df.loc[0]['crew'] #--> as a heading "name"

result_df.loc[0]['cast'] #--> as a heading "name"

result_df.loc[0]['keywords'] #--> as a heading "name"

result_df.loc[0]['genres'] #--> as a heading "name"


In [24]:
# get. the names list from all thses features

def get_name_list(common_value):
  name_list=[]

  if isinstance(common_value,list):# isinstance()-->checking whether given data is list
    for elememnt in common_value:
      name_list .append(elememnt['name']) 

  return name_list

In [25]:
# call the above function for all the 3 oparameters

features=['cast','keywords','genres']

for feature in features:
  result_df[feature] = result_df[feature].apply(get_name_list)

In [ ]:
result_df[['cast','director','keywords','genres']].head()

In [27]:
# remove all capslock and space diffrences in names

# cast --> is list
# director --> is string

def clean_data(coloumn_value):
  modified_list=[]
  modified_string=""

  if isinstance(coloumn_value,list):
    for element in coloumn_value:
      modified_string=element.replace(" ","")
      modified_list.append(modified_string.lower())
    return modified_list


  elif isinstance(coloumn_value,str):
    modified_string=coloumn_value.replace(" ","")
    return modified_string.lower()

  else:
    return ""

  



In [28]:
features=['cast','keywords','genres','director']

for feature in features:
  result_df[feature]=result_df[feature].apply(clean_data)



In [ ]:
result_df[['cast','director','keywords','genres']].head()

In [ ]:
# combine all the list values together
b=["hai,","how","are","you"]
c=" ".join(b)
print(c)


hai, how are you


In [30]:
def create_soup(x):
  return " ".join(x['keywords'])+" "+" ".join(x['cast'])+" "+" ".join(x['director'])+" "+" ".join(x['genres'])

In [31]:
result_df['soup']=result_df.apply(create_soup,axis=1)

In [32]:
result_df[['original_title' , 'soup']]

,original_title,soup
0,Avatar,cultureclash future spacewar spacecolony socie...
1,Pirates of the Caribbean: At World's End,ocean drugabuse exoticisland eastindiatradingc...
2,Spectre,spy basedonnovel secretagent sequel mi6 britis...
3,The Dark Knight Rises,dccomics crimefighter terrorist secretidentity...
4,John Carter,basedonnovel mars medallion spacetravel prince...
...,...,...
4795,Bang,gang audition policefake homeless actress darl...
4796,Primer,distrust garage identitycrisis timetravel time...
4798,El Mariachi,unitedstates–mexicobarrier legs arms paperknif...
4799,Newlyweds,edwardburns kerrybishé marshadietlein caitlin...


In [33]:
# copy paste : https://www.machinelearningplus.com/nlp/cosine-similarity/

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')

sparse_matrix = count_vectorizer.fit_transform(result_df['soup'])

from sklearn.metrics.pairwise import cosine_similarity
cosine_similiarity2=cosine_similarity(sparse_matrix,sparse_matrix)



In [35]:
# resetting index and recommenting movies
result_df = result_df.reset_index()
indices = pd.Series(result_df.index, index=result_df['original_title'])

def get_recommendations(title):
   idx = indices[title]
   sim_scores = list(enumerate(cosine_similiarity2[idx]))
   sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
   sim_scores = sim_scores[1:11]
   movie_indices = [i[0] for i in sim_scores]

   return result_df['original_title'].iloc[movie_indices]

In [37]:
get_recommendations("Fight Club")

1469                       Se7en
44                   World War Z
915                     The Game
411                       Zodiac
3637       Me You and Five Bucks
3867    Amidst the Devil's Wings
218                      RoboCop
2202                  Salton Sea
2862           Princess Kaiulani
3494                Sleep Dealer
Name: original_title, dtype: object

In [ ]:
result_df.to_csv('movies.csv')

In [ ]:
# downloading file
from google.colab import files
files.download('movies.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>